In [1]:
import findspark

findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [3]:
import pandas as pd

# 파일 불러오기
data = pd.read_csv('match_data.csv')

In [4]:
data.head()

,Unnamed: 0,date,title,stage,team1,team2,win,top1,jg1,mid1,adc1,sup1,top2,jg2,mid2,adc2,sup2
0,0,2021.08.29,2021 LCK 서머 포스트 시즌,결승전 4세트,T1,DWG KIA,L,T1 Canna,T1 Oner,T1 Faker,T1 Gumayusi,T1 Keria,DK Khan,DK Canyon,DK ShowMaker,DK Ghost,DK BeryL
1,1,2021.08.29,2021 LCK 서머 포스트 시즌,결승전 3세트,T1,DWG KIA,W,T1 Canna,T1 Oner,T1 Faker,T1 Gumayusi,T1 Keria,DK Khan,DK Canyon,DK ShowMaker,DK Ghost,DK BeryL
2,2,2021.08.29,2021 LCK 서머 포스트 시즌,결승전 2세트,DWG KIA,T1,W,DK Khan,DK Canyon,DK ShowMaker,DK Ghost,DK BeryL,T1 Canna,T1 Oner,T1 Faker,T1 Teddy,T1 Keria
3,3,2021.08.29,2021 LCK 서머 포스트 시즌,결승전 1세트,DWG KIA,T1,W,DK Khan,DK Canyon,DK ShowMaker,DK Ghost,DK BeryL,T1 Canna,T1 Oner,T1 Faker,T1 Teddy,T1 Keria
4,4,2021.08.22,2021 LCK 서머 포스트 시즌,2라운드 2경기 4세트,Gen.G eSports,T1,L,GEN Rascal,GEN Clid,GEN BDD,GEN Ruler,GEN Life,T1 Canna,T1 Oner,T1 Faker,T1 Teddy,T1 Keria


In [5]:
len(data)

477

In [7]:
# 컬럼 수정 & 생성

data['date'] = data['date'].apply(lambda x : x.replace('.','-'))
data['stage'] = data['stage'].apply(lambda x :x[:-4])  # '0세트' 제거
data['point1'] = 0   # point1, point2 컬럼 생성
data['point2'] = 0

In [8]:
# 경기당 세트 수 카운트

set_count = data.groupby(['title','stage']).date.count().reset_index()
set_count

,title,stage,date
0,2021 LCK 서머 포스트 시즌,1라운드 1경기,4
1,2021 LCK 서머 포스트 시즌,1라운드 2경기,3
2,2021 LCK 서머 포스트 시즌,2라운드 1경기,3
3,2021 LCK 서머 포스트 시즌,2라운드 2경기,4
4,2021 LCK 서머 포스트 시즌,결승전,4
...,...,...,...
185,2021 LoL 챔피언스 코리아 스프링 정규 시즌,7일차 2경기,2
186,2021 LoL 챔피언스 코리아 스프링 정규 시즌,8일차 1경기,3
187,2021 LoL 챔피언스 코리아 스프링 정규 시즌,8일차 2경기,3
188,2021 LoL 챔피언스 코리아 스프링 정규 시즌,9일차 1경기,2


In [9]:
# 이렇게 했어도 됐었ㅅ구나,,,^^;;;

#data.drop_duplicates(['title','stage'], keep = 'first')

In [10]:
set_count.date.unique()

array([4, 3, 5, 2], dtype=int64)

In [11]:
set_count.loc[(set_count.title=='2021 LCK 서머 포스트 시즌') & (set_count.stage=='1라운드 1경기'),'date'].values[0]

4

In [12]:
# 몇대몇인지 계산

index = 0
for i in range(len(set_count)):
    team = data.loc[index,'team1']
    title = data.loc[index,'title']
    stage = data.loc[index,'stage']
    count = int(set_count.loc[(set_count.title==title) & (set_count.stage==stage),'date'].values[0])
    for j in range(count):
        if data.loc[index+j,'team1'] == team :
            if data.loc[index+j,'win'] == 'W':
                data.loc[index,'point1'] += 1
            else:
                data.loc[index,'point2'] += 1
        else:
            if data.loc[index+j,'win'] == 'W':
                data.loc[index,'point2'] += 1
            else:
                data.loc[index,'point1'] += 1
    #print(team, title, stage, count, index)
        
    index += count

In [13]:
# 첫번째 데이터만 남김

data = data.loc[((data.point1!=0)|(data.point2!=0)),]
data = data.drop(['Unnamed: 0'], axis=1)
data

,date,title,stage,team1,team2,win,top1,jg1,mid1,adc1,sup1,top2,jg2,mid2,adc2,sup2,point1,point2
0,2021-08-29,2021 LCK 서머 포스트 시즌,결승전,T1,DWG KIA,L,T1 Canna,T1 Oner,T1 Faker,T1 Gumayusi,T1 Keria,DK Khan,DK Canyon,DK ShowMaker,DK Ghost,DK BeryL,1,3
4,2021-08-22,2021 LCK 서머 포스트 시즌,2라운드 2경기,Gen.G eSports,T1,L,GEN Rascal,GEN Clid,GEN BDD,GEN Ruler,GEN Life,T1 Canna,T1 Oner,T1 Faker,T1 Teddy,T1 Keria,1,3
8,2021-08-21,2021 LCK 서머 포스트 시즌,2라운드 1경기,NS RED FORCE,DWG KIA,L,NS Rich,NS Peanut,NS Gori,NS deokdam,NS Kellin,DK Khan,DK Canyon,DK ShowMaker,DK Ghost,DK BeryL,0,3
11,2021-08-19,2021 LCK 서머 포스트 시즌,1라운드 2경기,Liiv Sandbox,T1,L,LSB Summit,LSB Croco,LSB Fate,LSB Prince,LSB Effort,T1 Canna,T1 Oner,T1 Faker,T1 Teddy,T1 Keria,0,3
14,2021-08-18,2021 LCK 서머 포스트 시즌,1라운드 1경기,Afreeca Freecs,NS RED FORCE,L,AF Kiin,AF Dread,AF Fly,AF Leo,AF Lehends,NS Rich,NS Peanut,NS Gori,NS deokdam,NS Kellin,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
464,2021-01-15,2021 LoL 챔피언스 코리아 스프링 정규 시즌,3일차 1경기,Gen.G eSports,Fredit BRION,W,GEN Rascal,GEN Clid,GEN BDD,GEN Ruler,GEN Life,BRO Hoya,BRO UmTi,BRO Lava,BRO Hena,BRO Delight,2,0
466,2021-01-14,2021 LoL 챔피언스 코리아 스프링 정규 시즌,2일차 2경기,DRX,Afreeca Freecs,W,DRX Kingen,DRX Pyosik,DRX SOLKA,DRX BAO,DRX Becca,AF Kiin,AF Dread,AF Fly,AF Bang,AF Lehends,2,1
469,2021-01-14,2021 LoL 챔피언스 코리아 스프링 정규 시즌,2일차 1경기,NS RED FORCE,Liiv Sandbox,W,NS Rich,NS Peanut,NS Bay,NS deokdam,NS Kellin,LSB Summit,LSB Croco,LSB Fate,LSB Route,LSB Effort,2,1
472,2021-01-13,2021 LoL 챔피언스 코리아 스프링 정규 시즌,1일차 2경기,T1,Hanwha Life Esports,W,T1 Canna,T1 Ellim,T1 Clozer,T1 Gumayusi,T1 Keria,HLE Morgan,HLE Arthur,HLE Chovy,HLE Deft,HLE Vsta,2,1


In [14]:
# hdfs에 저장하기

df = spark.createDataFrame(data)
df

DataFrame[date: string, title: string, stage: string, team1: string, team2: string, win: string, top1: string, jg1: string, mid1: string, adc1: string, sup1: string, top2: string, jg2: string, mid2: string, adc2: string, sup2: string, point1: bigint, point2: bigint]

In [15]:
df.show(5)

+----------+-------------------------+-------------+--------------+------------+---+----------+---------+--------+-----------+----------+--------+---------+------------+----------+---------+------+------+
|      date|                    title|        stage|         team1|       team2|win|      top1|      jg1|    mid1|       adc1|      sup1|    top2|      jg2|        mid2|      adc2|     sup2|point1|point2|
+----------+-------------------------+-------------+--------------+------------+---+----------+---------+--------+-----------+----------+--------+---------+------------+----------+---------+------+------+
|2021-08-29|2021 LCK 서머 포스트 시즌|       결승전|            T1|     DWG KIA|  L|  T1 Canna|  T1 Oner|T1 Faker|T1 Gumayusi|  T1 Keria| DK Khan|DK Canyon|DK ShowMaker|  DK Ghost| DK BeryL|     1|     3|
|2021-08-22|2021 LCK 서머 포스트 시즌|2라운드 2경기| Gen.G eSports|          T1|  L|GEN Rascal| GEN Clid| GEN BDD|  GEN Ruler|  GEN Life|T1 Canna|  T1 Oner|    T1 Faker|  T1 Teddy| T1 Keria|     1|     

In [18]:
df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("hdfs://localhost:19000/data/match_data_2.csv")

In [19]:
temp = spark.read.csv("hdfs://localhost:19000/data/match_data_2.csv", header="true", inferSchema="true")

In [20]:
temp.show(5)

+----------+-------------------------+-------------+--------------+------------+---+----------+---------+--------+-----------+----------+--------+---------+------------+----------+---------+------+------+
|      date|                    title|        stage|         team1|       team2|win|      top1|      jg1|    mid1|       adc1|      sup1|    top2|      jg2|        mid2|      adc2|     sup2|point1|point2|
+----------+-------------------------+-------------+--------------+------------+---+----------+---------+--------+-----------+----------+--------+---------+------------+----------+---------+------+------+
|2021-08-29|2021 LCK 서머 포스트 시즌|       결승전|            T1|     DWG KIA|  L|  T1 Canna|  T1 Oner|T1 Faker|T1 Gumayusi|  T1 Keria| DK Khan|DK Canyon|DK ShowMaker|  DK Ghost| DK BeryL|     1|     3|
|2021-08-22|2021 LCK 서머 포스트 시즌|2라운드 2경기| Gen.G eSports|          T1|  L|GEN Rascal| GEN Clid| GEN BDD|  GEN Ruler|  GEN Life|T1 Canna|  T1 Oner|    T1 Faker|  T1 Teddy| T1 Keria|     1|     